# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Выводы шага 1:

* У нас есть база в 3214 строк и 5 столбцов.
* Пропусков в данных нет, типы данных и наименования корректные.

## Разбейте данные на выборки

In [5]:
# делим все данные на паризнаки и целевой признак 
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

# разбиваем данные на обучающую, валидационную и тестовую выборки

# для начала выберем обучающую выборку в размере 60%

features_train, features_x, target_train, target_x = train_test_split(
    features, target, test_size=0.4, random_state=12345) 


# из оставшейся выборки выберем половину на валидационные и тестовые данные

features_valid, features_test,  target_valid, target_test = train_test_split(
    features_x, target_x, test_size=0.5, random_state=12345)

In [6]:
# проверим выборки на размерность

print('features:', features.shape)
print('target:', target.shape)

print('features_train:', features_train.shape)
print('target_train:', target_train.shape)

print('features_valid:', features_valid.shape)
print('target_valid:', target_valid.shape)

print('features_test:', features_test.shape)
print('target_test:', target_test.shape)

features: (3214, 4)
target: (3214,)
features_train: (1928, 4)
target_train: (1928,)
features_valid: (643, 4)
target_valid: (643,)
features_test: (643, 4)
target_test: (643,)


Выводы шага 2:

* Разделили базу на признаки и целевой признак
* Разделили базу на обучающую, валидационную и тестовую выборки в соотношении 60/20/20
* Проверили размерность полученных выборок

## Исследуйте модели

### Модель дерева решений

Изучим модель дерева решений и сразу выберем глубину для наилучшего решения.

In [8]:
# исследуем глубину дерева до 10

max_accuracy = 0
best_depth = 0

for depth in range(1, 11):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train , target_train)

    predictions_valid = model_tree.predict(features_valid)

    print("max_depth =", depth, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))
    
    if accuracy_score(target_valid, predictions_valid) > max_accuracy:
        max_accuracy = accuracy_score(target_valid, predictions_valid)
        best_depth = depth
        model_tree_best = model_tree
          
print(f'Максимальное значение accuracy при данной модели решения:{max_accuracy}, max_depth: {best_depth}')

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944
max_depth = 6 : 0.7838258164852255
max_depth = 7 : 0.7822706065318819
max_depth = 8 : 0.7791601866251944
max_depth = 9 : 0.7822706065318819
max_depth = 10 : 0.7744945567651633
Максимальное значение accuracy при данной модели решения:0.7853810264385692, max_depth: 3


Вывод:  Для модели дерева решений (DecisionTreeClassifier) наилушим показателем accuracy оказалось 0.7853810264385692 при max_depth = 3

### Модель Случайный лес

Изучим модель Случайный лес и сразу выберем количество деревьев для наилучшего решения.

In [9]:
best_result = 0
best_est = 0

for est in range(1, 21):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators= est)
    
    model_forest.fit(features_train, target_train)
    result = model_forest.score(features_valid, target_valid)
    
    if result > best_result:
        best_result = result
        best_est = est
        model_forest_best = model_forest

print(f'Accuracy наилучшей модели на валидационной выборке:{best_result}, количество деревьев {best_est}')

Accuracy наилучшей модели на валидационной выборке:0.7931570762052877, количество деревьев 18


Вывод: Для модели Случайный лес (RandomForestClassifier) наилушим показателем accuracy оказалось 0.7931570762052877 при n_estimators = 18

### Модель Логистическая регрессия

Изучим модель Логистической регрессии.

In [10]:
model_log = LogisticRegression(random_state=12345) 
model_log.fit(features_train, target_train)

result = model_log.score(features_valid, target_valid)

result

0.7107309486780715

Вывод: Для модели Логистической регрессии (LogisticRegressionClassifier)  accuracy равна 0.7107309486780715

Общий вывод по шагу 3:

Мы применили три модели для задачи классификации - дерево решений с глубиной от 1 до 10, случайный лес с количеством деревьев от 1 до 20 и логическую регрессию.

Для модели дерева решений (DecisionTreeClassifier) наилушим показателем accuracy оказалось 0.7853810264385692 при max_depth = 3

Для модели Случайный лес (RandomForestClassifier) наилушим показателем accuracy оказалось 0.7931570762052877 при n_estimators = 18

Для модели Логистической регрессии (LogisticRegressionClassifier) мы получили accuracy равной 0.7107309486780715

## Проверьте модель на тестовой выборке

Выберем модель с лучшим показателем accuracy  - Случайный лес с количеством деревьев 18. Посчитаем accuracy на тестовой выборке.

In [11]:
model_forest_best.score(features_test, target_test)

0.7853810264385692

Выбрав лучший показатель accuracy - Случайный лес с количеством деревьев 18, мы протестировали модель на тестовой выборке и получили значение accuracy = 0.7853810264385692

## (бонус) Проверьте модели на адекватность

In [96]:
model_dc = DummyClassifier(strategy='most_frequent', random_state=12345)
model_dc.fit(features_train, target_train)
result_dc = model_dc.score(features_valid, target_valid)
print('Accuracy DummyClassifier:', result_dc)

Accuracy DummyClassifier: 0.7060653188180405


In [97]:
# is_ultra = 1

target.sum()

985

In [100]:
# is_ultra = 1

len(target) - target.sum()

2229

In [102]:
# процент распространенности популярного класса в исходных данных

(len(target) - target.sum())/len(target)

0.693528313627878

In [104]:
# лучшая модель на общих данных сделает прогноз целевого признака

pred_total = model_forest_best.predict(features)

In [107]:
# найдем сумму значений is_ultra = 1

sum(pred_total)

868

In [109]:
# процент распространенности популярного класса в прогнозируемых данных

(len(pred_total) - sum(pred_total))/len(pred_total)

0.7299315494710641

## Общий вывод

Шаг 1.

* У нас есть база в 3214 строк и 5 столбцов.
* Пропусков в данных нет, типы данных и наименования корректные.

Шаг 2.

* Разделили базу на признаки и целевой признак
* Разделили базу на обучающую, валидационную и тестовую выборки в соотношении 60/20/20
* Проверили размерность полученных выборок

Шаг 3.

Мы применили три модели для задачи классификации - дерево решений с глубиной от 1 до 10, случайный лес с количеством деревьев от 1 до 20 и логическую регрессию.

* Для модели дерева решений (DecisionTreeClassifier) наилушим показателем accuracy оказалось 0.7853810264385692 при max_depth = 3

* Для модели Случайный лес (RandomForestClassifier) наилушим показателем accuracy оказалось 0.7931570762052877 при n_estimators = 18

* Для модели Логистической регрессии (LogisticRegressionClassifier) мы получили accuracy равной 0.7107309486780715

Шаг4.

Выбрав лучший показатель accuracy - Случайный лес с количеством деревьев 18, мы протестировали модель на тестовой выборке и получили значение accuracy = 0.7853810264385692
